# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
import os

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,medford,42.3265,-122.8756,8.85,79,75,0.00,US,1670287671
1,1,hobart,-42.8794,147.3294,16.70,56,75,8.23,AU,1670287505
2,2,georgetown,5.4112,100.3354,27.35,92,20,1.03,MY,1670287415
3,3,busselton,-33.6500,115.3333,19.35,44,0,5.67,AU,1670287672
4,4,isangel,-19.5500,169.2667,31.06,74,20,1.54,VU,1670287672


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",s="Humidity",c="City",alpha=0.6)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
weather_condition_df = city_data_df
weather_condition_df = weather_condition_df.loc[(weather_condition_df["Max Temp"] < 27) & (weather_condition_df["Max Temp"] > 21)]
weather_condition_df = weather_condition_df.loc[weather_condition_df["Wind Speed"] < 4.5]
weather_condition_df = weather_condition_df.loc[weather_condition_df["Cloudiness"] == 0]


# Drop any rows with null values
weather_condition_df = weather_condition_df.dropna()

# Display sample data
weather_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
77,77,zihuatanejo,17.6333,-101.5500,25.81,80,0,0.61,MX,1670287627
153,153,salalah,17.0151,54.0924,22.05,60,0,2.57,OM,1670287645
157,157,san juan del rio,20.3833,-100.0000,21.96,35,0,1.59,MX,1670287716
168,168,sur,22.5667,59.5289,22.19,62,0,1.57,OM,1670287719
172,172,puerto escondido,15.8500,-97.0667,25.29,67,0,1.35,MX,1670287720
307,307,shreveport,32.5251,-93.7502,23.12,78,0,4.12,US,1670287660
401,401,cayenne,4.9333,-52.3333,25.02,94,0,0.00,GF,1670287786
422,422,montes claros,-16.7350,-43.8617,21.88,64,0,1.54,BR,1670287575
427,427,umm lajj,25.0213,37.2685,22.80,73,0,2.57,SA,1670287794
506,506,saint-pierre,-21.3393,55.4781,22.82,64,0,3.09,RE,1670287819


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_condition_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = pd.Series(dtype="str")

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
77,77,zihuatanejo,17.6333,-101.5500,25.81,80,0,0.61,MX,1670287627,NaN
153,153,salalah,17.0151,54.0924,22.05,60,0,2.57,OM,1670287645,NaN
157,157,san juan del rio,20.3833,-100.0000,21.96,35,0,1.59,MX,1670287716,NaN
168,168,sur,22.5667,59.5289,22.19,62,0,1.57,OM,1670287719,NaN
172,172,puerto escondido,15.8500,-97.0667,25.29,67,0,1.35,MX,1670287720,NaN
307,307,shreveport,32.5251,-93.7502,23.12,78,0,4.12,US,1670287660,NaN
401,401,cayenne,4.9333,-52.3333,25.02,94,0,0.00,GF,1670287786,NaN
422,422,montes claros,-16.7350,-43.8617,21.88,64,0,1.54,BR,1670287575,NaN
427,427,umm lajj,25.0213,37.2685,22.80,73,0,2.57,SA,1670287794,NaN
506,506,saint-pierre,-21.3393,55.4781,22.82,64,0,3.09,RE,1670287819,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 3
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
zihuatanejo - nearest hotel: Embarc Zihuatanejo
salalah - nearest hotel: Muscat International Hotel
san juan del rio - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
puerto escondido - nearest hotel: Hotel Barlovento
shreveport - nearest hotel: Sam's Town Hotel and Casino Shreveport
cayenne - nearest hotel: Central Hôtel
montes claros - nearest hotel: Hotel Alvorada
umm lajj - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
77,77,zihuatanejo,17.6333,-101.5500,25.81,80,0,0.61,MX,1670287627,Embarc Zihuatanejo
153,153,salalah,17.0151,54.0924,22.05,60,0,2.57,OM,1670287645,Muscat International Hotel
157,157,san juan del rio,20.3833,-100.0000,21.96,35,0,1.59,MX,1670287716,No hotel found
168,168,sur,22.5667,59.5289,22.19,62,0,1.57,OM,1670287719,Sur Hotel
172,172,puerto escondido,15.8500,-97.0667,25.29,67,0,1.35,MX,1670287720,Hotel Barlovento
307,307,shreveport,32.5251,-93.7502,23.12,78,0,4.12,US,1670287660,Sam's Town Hotel and Casino Shreveport
401,401,cayenne,4.9333,-52.3333,25.02,94,0,0.00,GF,1670287786,Central Hôtel
422,422,montes claros,-16.7350,-43.8617,21.88,64,0,1.54,BR,1670287575,Hotel Alvorada
427,427,umm lajj,25.0213,37.2685,22.80,73,0,2.57,SA,1670287794,No hotel found
506,506,saint-pierre,-21.3393,55.4781,22.82,64,0,3.09,RE,1670287819,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:

%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",s="Humidity",c="City",alpha=0.6,hover_cols=["Hotel Name","Country"])

# Display the map
map_hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)